In [1]:
import torch
from PIL import Image
from torchvision import transforms, models
from ModelAnalysis import ModelStatsAggregator, ModelStatAnalyser
from sys import version
import timm
from tqdm import tqdm
import urllib
from PIL import Image
from timm.data import resolve_data_config
from timm.data.transforms_factory import create_transform
version

'3.9.5 (default, Aug 29 2021, 19:01:31) \n[GCC 9.3.0]'

In [2]:
model_dict = {
    'ssd': torch.hub.load('NVIDIA/DeepLearningExamples:torchhub', 'nvidia_ssd'),
    'lenet': torch.hub.load('pytorch/vision:v0.10.0', 'googlenet'),
    'yolov5s': torch.hub.load('ultralytics/yolov5', 'yolov5s'),
    'yolov5m': torch.hub.load('ultralytics/yolov5', 'yolov5m'),
    'yolov5l': torch.hub.load('ultralytics/yolov5', 'yolov5l'),
    'yolov5x': torch.hub.load('ultralytics/yolov5', 'yolov5x'),
    'alexnet': models.alexnet(),
    'squeezenet_1_0': models.squeezenet1_1(),
    'squeezenet_1_1': models.squeezenet1_0(),
    'googlenet': models.googlenet(),
    'shufflenet_v2_x0_5': models.shufflenet_v2_x0_5(),
    'shufflenet_v2_x1_0': models.shufflenet_v2_x1_0(),
}

Using cache found in /home/sultan/.cache/torch/hub/NVIDIA_DeepLearningExamples_torchhub
Using cache found in /home/sultan/.cache/torch/hub/pytorch_vision_v0.10.0
/home/sultan/CNN_Network_Analysis/.venv/lib/python3.9/site-packages/torchvision/models/googlenet.py:77: FutureWarning: The default weight initialization of GoogleNet will be changed in future releases of torchvision. If you wish to keep the old behavior (which leads to long initialization times due to scipy/scipy#11299), please set init_weights=True.
  warnings.warn('The default weight initialization of GoogleNet will be changed in future releases of '
Using cache found in /home/sultan/.cache/torch/hub/ultralytics_yolov5_master
YOLOv5 🚀 2021-8-29 torch 1.9.0+cu102 CUDA:0 (GeForce RTX 2070 SUPER, 7979.1875MB)

Fusing layers... 
Model Summary: 224 layers, 7266973 parameters, 0 gradients
Adding AutoShape... 
Using cache found in /home/sultan/.cache/torch/hub/ultralytics_yolov5_master
YOLOv5 🚀 2021-8-29 torch 1.9.0+cu102 CUDA:0 (G

In [3]:
# prepare sample inputs
import urllib
url, filename = ("https://github.com/pytorch/hub/raw/master/images/dog.jpg", "dog.jpg")
try: urllib.URLopener().retrieve(url, filename)
except: urllib.request.urlretrieve(url, filename)

input_image = Image.open(filename)
preprocess = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])
input_tensor = preprocess(input_image)
input_batch = input_tensor.unsqueeze(0) # create a mini-batch as expected by the model

uris = [
    'http://images.cocodataset.org/val2017/000000397133.jpg'
]
utils = torch.hub.load('NVIDIA/DeepLearningExamples:torchhub', 'nvidia_ssd_processing_utils')

inputs = [utils.prepare_input(uri) for uri in uris]
ssd_input_batch = utils.prepare_tensor(inputs)

Using cache found in /home/sultan/.cache/torch/hub/NVIDIA_DeepLearningExamples_torchhub


In [4]:
stats_dict, _ = ModelStatAnalyser.get_models_stats_dict(model_dict, input_batch, ssd_input_batch) 

Analysing ssd
Analysing lenet
Analysing yolov5s
Analysing yolov5m
Analysing yolov5l
Analysing yolov5x
Analysing alexnet
Analysing squeezenet_1_0
Analysing squeezenet_1_1
Analysing googlenet
Analysing shufflenet_v2_x0_5
Analysing shufflenet_v2_x1_0


In [5]:
model_batch_size = 4
model_list = timm.list_models(exclude_filters=['*_iabn', 'levit_*', 'tresnet_*'],pretrained=False)
in_shapes = []
for model_batch_idx in tqdm(range(0, len(model_list), model_batch_size)):
    for model_name in model_list[model_batch_idx: model_batch_idx+model_batch_size]:
        model = timm.create_model(model_name, pretrained=False)
        config = resolve_data_config({}, model=model)
        transform = create_transform(**config)

        url, filename = ("https://github.com/pytorch/hub/raw/master/images/dog.jpg", "dog.jpg")
        urllib.request.urlretrieve(url, filename)
        img = Image.open(filename).convert('RGB')
        tensor = transform(img).unsqueeze(0) # transform and add batch dimension
        in_tensor_shape = tensor.size()
        if in_tensor_shape[2] == 224 and in_tensor_shape[3] == 224:       
            in_shapes.append(in_tensor_shape)
            model_dict = {}
            model_dict[model_name] = model
            res_stats_dict, _ = ModelStatAnalyser.get_models_stats_dict(model_dict, tensor, ssd_input_batch)
            stats_dict = stats_dict | res_stats_dict

  0%|          | 0/149 [00:00<?, ?it/s]/home/sultan/CNN_Network_Analysis/.venv/lib/python3.9/site-packages/torchvision/transforms/transforms.py:280: UserWarning: Argument interpolation should be of type InterpolationMode instead of int. Please, use InterpolationMode enum.
  warnings.warn(
  1%|          | 1/149 [00:01<03:31,  1.43s/it]

Analysing cait_s24_224


  1%|▏         | 2/149 [00:08<11:34,  4.73s/it]

Analysing cait_xxs24_224


  2%|▏         | 3/149 [00:10<08:53,  3.66s/it]

...

 98%|█████████▊| 146/149 [07:30<00:13,  4.55s/it]

Analysing vit_tiny_patch16_224_in21k
Analysing vit_tiny_r_s16_p8_224
Analysing vit_tiny_r_s16_p8_224_in21k


 99%|█████████▊| 147/149 [07:31<00:07,  3.55s/it]

Analysing vovnet39a
Analysing vovnet57a
Analysing wide_resnet50_2
Analysing wide_resnet101_2


100%|██████████| 149/149 [07:35<00:00,  3.06s/it]


In [12]:
total_networks = len(stats_dict.keys())
networks_with_kernel = {f'{k}': 0 for k in range(1, 12)}
for network, stats in stats_dict.items():
    for kernel_size in networks_with_kernel.keys():
        if kernel_size in stats['kernel'].keys():
            networks_with_kernel[kernel_size] += 1


413


In [13]:
stats_dict['alexnet']

{'kernel': {'11': 1, '5': 1, '3': 3},
 'stride': {'11': {'4': 1}, '5': {'1': 1}, '3': {'1': 3}},
 'in_channel': {'11': {'3': 1},
  '5': {'64': 1},
  '3': {'192': 1, '384': 1, '256': 1}},
 'filters': {'11': {'64': 1}, '5': {'192': 1}, '3': {'384': 1, '256': 2}},
 'intermediate_layer_sizes': [(150528, [1, 3, 224, 224]),
  (46656, [1, 64, 27, 27]),
  (32448, [1, 192, 13, 13]),
  (64896, [1, 384, 13, 13]),
  (43264, [1, 256, 13, 13]),
  (43264, [256, 13, 13])],
 'intermediate_layer_bounds': ((150528, [1, 3, 224, 224]),
  (32448, [1, 192, 13, 13]))}

In [7]:
ModelStatsAggregator.get_aggregate_kernel_stats_as_percentages(stats_dict)

{'7': 0.008996066358816487,
 '1': 0.6101932615016248,
 '3': 0.35016247648366683,
 '11': 0.0001026167265264238,
 '5': 0.020489139729775955,
 '16': 0.001573456473405165,
 '4': 0.0012998118693347016,
 '2': 0.003728407730460065,
 '32': 0.0003078501795792714,
 '9': 0.0024628014366341712,
 '14': 0.0001026167265264238,
 '8': 0.000581494783649735}

In [40]:
expected_kernel_sizes = list(range(1,12))
count_of_models_with_kernel_size = {f'{k}': 0 for k in expected_kernel_sizes}
for model, stats in stats_dict.items():
    for size in expected_kernel_sizes:
        count_of_models_with_kernel_size[f'{size}'] += (1 if stats['kernel'][f'{size}'] > 0 else 0)
print(count_of_models_with_kernel_size)

{'1': 380, '2': 22, '3': 359, '4': 22, '5': 66, '6': 46, '7': 156, '8': 11, '9': 8, '10': 0, '11': 0}


In [8]:
ModelStatsAggregator.get_aggregate_stride_stats_per_kernel(stats_dict)

{'7': {'2': 151, '1': 107, '4': 2},
 '1': {'1': 17371, '2': 469},
 '3': {'1': 8602, '2': 1636},
 '11': {'4': 1, '2': 2},
 '5': {'1': 443, '2': 155},
 '16': {'16': 40, '8': 6},
 '4': {'4': 38},
 '2': {'2': 109},
 '32': {'32': 9},
 '9': {'2': 13, '1': 60},
 '14': {'7': 2, '14': 1},
 '8': {'8': 17}}

In [9]:
ModelStatsAggregator.get_aggregate_in_channel_stats_per_kernel(stats_dict)

{'7': {'3': 135,
  '120': 16,
  '90': 10,
  '300': 10,
  '48': 9,
  '512': 8,
  '192': 7,
  '156': 6,
  '396': 6,
  '144': 6,
  '57': 5,
  '80': 5,
  '288': 5,
  '480': 4,
  '216': 4,
  '720': 4,
  '24': 3,
  '81': 3,
  '112': 3,
  '64': 2,
  '60': 2,
  '240': 2,
  '180': 2,
  '160': 2,
  '96': 1,
  '72': 1,
  '128': 1,
  '432': 1},
 '1': {'1024': 2824,
  '256': 2166,
  '512': 1844,
  '64': 1196,
  '128': 852,
  '2048': 508,
  '32': 421,
  '192': 312,
  '768': 289,
  '384': 282,
  '240': 251,
  '96': 241,
  '16': 231,
  '480': 209,
  '144': 205,
  '80': 202,
  '1536': 196,
  '40': 193,
  '120': 186,
  '48': 185,
  '24': 185,
  '672': 182,
  '72': 169,
  '160': 167,
  '1152': 162,
  '576': 155,
  '320': 152,
  '640': 139,
  '896': 138,
  '112': 131,
  '288': 116,
  '336': 105,
  '20': 98,
  '960': 91,
  '448': 90,
  '4096': 83,
  '8': 81,
  '168': 75,
  '224': 74,
  '432': 72,
  '720': 68,
  '28': 59,
  '60': 51,
  '1344': 49,
  '528': 48,
  '416': 47,
  '832': 44,
  '56': 44,
  '208': 

In [15]:
ModelStatsAggregator.get_aggregate_filter_stats_per_kernel(stats_dict)

{'7': {'64': 110,
  '120': 16,
  '16': 13,
  '90': 10,
  '300': 10,
  '48': 9,
  '192': 9,
  '4096': 8,
  '156': 6,
  '396': 6,
  '144': 6,
  '96': 5,
  '57': 5,
  '128': 5,
  '80': 5,
  '288': 5,
  '24': 4,
  '480': 4,
  '216': 4,
  '720': 4,
  '81': 3,
  '112': 3,
  '60': 2,
  '240': 2,
  '180': 2,
  '160': 2,
  '72': 1,
  '432': 1,
  '256': 1,
  '40': 1,
  '32': 1},
 '1': {'1024': 2753,
  '256': 2199,
  '512': 1954,
  '128': 1275,
  '64': 950,
  '2048': 724,
  '192': 482,
  '32': 353,
  '768': 280,
  '240': 237,
  '96': 235,
  '384': 231,
  '16': 223,
  '40': 219,
  '48': 206,
  '80': 200,
  '144': 187,
  '1536': 185,
  '24': 184,
  '480': 181,
  '120': 170,
  '672': 159,
  '160': 154,
  '576': 154,
  '1152': 152,
  '72': 137,
  '112': 133,
  '288': 128,
  '640': 124,
  '896': 121,
  '320': 116,
  '20': 99,
  '336': 95,
  '4096': 85,
  '168': 77,
  '8': 74,
  '448': 72,
  '1280': 68,
  '960': 64,
  '720': 62,
  '432': 62,
  '224': 61,
  '1056': 59,
  '28': 59,
  '60': 57,
  '56': 44

In [16]:
ModelStatsAggregator.get_aggregate_filter_stats(stats_dict)

{'256': 3839,
 '1024': 3241,
 '512': 2899,
 '128': 2245,
 '64': 1829,
 '32': 1106,
 '192': 847,
 '2048': 841,
 '48': 582,
 '768': 499,
 '384': 458,
 '96': 425,
 '240': 388,
 '120': 381,
 '144': 347,
 '160': 334,
 '16': 321,
 '80': 321,
 '72': 321,
 '40': 313,
 '480': 287,
 '320': 277,
 '672': 244,
 '1152': 233,
 '24': 232,
 '640': 218,
 '576': 206,
 '104': 206,
 '1536': 198,
 '112': 197,
 '288': 195,
 '896': 166,
 '224': 163,
 '18': 151,
 '60': 143,
 '36': 139,
 '336': 134,
 '4096': 125,
 '88': 119,
 '20': 109,
 '30': 109,
 '168': 107,
 '56': 105,
 '720': 104,
 '208': 104,
 '52': 103,
 '432': 96,
 '44': 96,
 '28': 95,
 '960': 93,
 '448': 91,
 '1280': 86,
 '176': 83,
 '8': 80,
 '26': 71,
 '152': 67,
 '1056': 66,
 '216': 61,
 '1344': 61,
 '528': 59,
 '6144': 54,
 '1392': 52,
 '368': 52,
 '300': 46,
 '3072': 46,
 '560': 46,
 '1232': 45,
 '624': 41,
 '90': 40,
 '10': 40,
 '184': 40,
 '800': 40,
 '304': 39,
 '352': 36,
 '416': 35,
 '156': 33,
 '100': 33,
 '1088': 31,
 '6': 31,
 '360': 31,
 